# scqubits example: Jaynes-Cummings model
J. Koch and P. Groszkowski

For further documentation of scqubits see https://scqubits.readthedocs.io/en/latest/.

---

In [2]:
import numpy as np

import scqubits as scq
from scqubits import HilbertSpace, InteractionTerm, ParameterSweep

## The Jaynes-Cummings model

The Jaynes-Cummings mode describes a system composed of a qubit (two-level system) coupled to a harmonic oscillator. The coupling is such that a qubit excitation can be "converted" into a photon and vice versa,

$\displaystyle H=-\frac{\omega_q}{2}\sigma_z + \omega_r a^\dagger a + g (a\sigma_+ + a^\dagger \sigma_-)$

This model can be solved exactly, resulting in the following eigenenergies:

$\displaystyle E_0 = -\frac{1}{2}\omega_q$,

$\displaystyle E_{n\pm} =  \bigg(n-\frac{1}{2}\bigg) \omega_r \pm \sqrt{\frac{\Delta^2}{4}+n g^2}$,

where $\Delta=\omega_r-\omega_q$ and $n=1,2,\ldots$.


In [3]:
# exact eigenenergies for comparing with numerics


def energies(frequency_r, frequency_q, g, n_cutoff):
    delta = frequency_r - frequency_q
    energies1 = (np.arange(1, n_cutoff) - 0.5) * frequency_r
    energies2 = np.sqrt(delta**2 / 4 + np.arange(1, n_cutoff) * g**2)
    energies_plus = energies1 + energies2
    energies_minus = energies1 - energies2
    energies_0 = np.asarray([[-0.5 * frequency_q]])
    all_energies = np.append(energies_0, energies_minus)
    all_energies = np.append(all_energies, energies_plus)
    return np.sort(all_energies)

### Set up the subsystems

In [5]:
frequency_q = 3.0
frequency_r = 5.0
g = 0.1

qubit1 = scq.GenericQubit(E=frequency_q)
qubit2 = scq.GenericQubit(E=frequency_q)
osc = scq.Oscillator(E_osc=frequency_r, truncated_dim=8)  # up to 9 photons

### Set up Hilbert space

In [6]:
# Form a list of all components making up the Hilbert space.
hilbertspace = HilbertSpace([qubit1, qubit2, osc])

hilbertspace.add_interaction(
    g_strength=g, op1=qubit1.sm_operator, op2=osc.creation_operator, add_hc=True
)

hilbertspace.add_interaction(
    g_strength=g, op1=qubit2.sm_operator, op2=osc.creation_operator, add_hc=True
)

hilbertspace.generate_lookup()

In [7]:
print(hilbertspace)

HilbertSpace:  subsystems
-------------------------

GenericQubit--------| [GenericQubit_3]
                    | E: 3.0
                    |
                    | dim: 2


GenericQubit--------| [GenericQubit_4]
                    | E: 3.0
                    |
                    | dim: 2


Oscillator----------| [Oscillator_2]
                    | E_osc: 5.0
                    | l_osc: None
                    | truncated_dim: 8
                    |
                    | dim: 8



HilbertSpace:  interaction terms
--------------------------------
InteractionTerm----------| [Interaction_1]
                         | g_strength: 0.1
                         | operator_list: [(0, <bound method GenericQubit.sm_operator of Gen ...
                         | add_hc: True

InteractionTerm----------| [Interaction_2]
                         | g_strength: 0.1
                         | operator_list: [(1, <bound method GenericQubit.sm_operator of Gen ...
                         | add_hc: 

In [8]:
dressed_hamiltonian = hilbertspace.hamiltonian()
dressed_hamiltonian

Quantum object: dims = [[2, 2, 8], [2, 2, 8]], shape = (32, 32), type = oper, isherm = True
Qobj data =
[[-3.  0.  0. ...  0.  0.  0.]
 [ 0.  2.  0. ...  0.  0.  0.]
 [ 0.  0.  7. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 28.  0.  0.]
 [ 0.  0.  0. ...  0. 33.  0.]
 [ 0.  0.  0. ...  0.  0. 38.]]

In [ ]:
evals = hilbertspace.eigenvals(evals_count=2)
print(evals)

[-0.5         0.25857864]


In [ ]:
energies(frequency_r, frequency_q, g, 3)

array([-0.5       ,  0.25857864,  0.54142136,  1.02679492,  1.37320508])

In [ ]:
import numpy as np
from qutip import Options
import qutip as qt
from quantum_logical.pulsesim import QuantumSystem, Pulse
from quantum_logical.pulsesim.hamiltonian import QubitQubitCavity
from quantum_logical.pulsesim.mode import QubitMode, CavityMode
import matplotlib.pyplot as plt

opts = Options(nsteps=1e6, atol=1e-8, rtol=1e-6)
p_bar = qt.ui.TextProgressBar()  # None